In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,21625
2,Huelva,Confirmados PDIA 14 días,5567
3,Huelva,Tasa PDIA 14 días,"1084,8256912913848"
4,Huelva,Confirmados PDIA 7 días,2383
5,Huelva,Total Confirmados,21851
6,Huelva,Curados,11699
7,Huelva,Fallecidos,189


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  21625.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5780.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 245 personas en los últimos 7 días 

Un positivo PCR cada 106 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,21625.0,2383.0,5567.0,513170.0,464.368533,1084.825691,206.0
Huelva-Costa,13146.0,1435.0,3594.0,289548.0,495.600039,1241.244975,130.0
Condado-Campiña,5924.0,712.0,1490.0,156231.0,455.735417,953.715972,65.0
Huelva (capital),5780.0,588.0,1353.0,143837.0,408.796068,940.648095,56.0
Ayamonte,1012.0,155.0,420.0,21104.0,734.457923,1990.144049,23.0
Lepe,1739.0,196.0,466.0,27880.0,703.012912,1671.449067,17.0
Bollullos Par del Condado,475.0,102.0,161.0,14387.0,708.973379,1119.065823,16.0
Sierra de Huelva-Andévalo Central,2203.0,153.0,380.0,67391.0,227.033283,563.873514,9.0
Palma del Condado (La),898.0,86.0,184.0,10801.0,796.222572,1703.545968,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,410.0,38.0,128.0,3366.0,1128.936423,3802.733214,2.0
Paymogo,60.0,10.0,31.0,1162.0,860.585198,2667.814114,2.0
Berrocal,14.0,2.0,8.0,303.0,660.066007,2640.264026,0.0
Isla Cristina,1788.0,171.0,560.0,21393.0,799.326883,2617.678680,5.0
San Bartolomé de la Torre,172.0,35.0,86.0,3761.0,930.603563,2286.625897,3.0
Ayamonte,1012.0,155.0,420.0,21104.0,734.457923,1990.144049,23.0
Cabezas Rubias,26.0,8.0,13.0,706.0,1133.144476,1841.359773,0.0
Palma del Condado (La),898.0,86.0,184.0,10801.0,796.222572,1703.545968,8.0
Lepe,1739.0,196.0,466.0,27880.0,703.012912,1671.449067,17.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,197.0,2.0,30.0,4602.0,43.459365,651.890482,0.0,0.066667
Higuera de la Sierra,29.0,1.0,8.0,1291.0,77.459334,619.674671,0.0,0.125000
Escacena del Campo,118.0,2.0,12.0,2287.0,87.450809,524.704854,0.0,0.166667
Campillo (El),62.0,1.0,4.0,2024.0,49.407115,197.628458,0.0,0.250000
Berrocal,14.0,2.0,8.0,303.0,660.066007,2640.264026,0.0,0.250000
Puebla de Guzmán,68.0,4.0,15.0,3092.0,129.366106,485.122898,1.0,0.266667
Villalba del Alcor,410.0,38.0,128.0,3366.0,1128.936423,3802.733214,2.0,0.296875
Isla Cristina,1788.0,171.0,560.0,21393.0,799.326883,2617.678680,5.0,0.305357
Manzanilla,79.0,7.0,22.0,2118.0,330.500472,1038.715770,1.0,0.318182
